# Avaliando o modelo base com o MMLU

In [ ]:
!pip install -U -q fsspec datasets transformers huggingface_hub bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
mmlu_dataset = load_dataset("cais/mmlu", "all")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.50M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/408k [00:00<?, ?B/s]

...

auxiliary_train-00000-of-00001.parquet:   0%|          | 0.00/47.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

Pegando 150 questões do MMLU de teste. Importante dizer que com a seed 21, sempre saíra as mesmas questões

In [ ]:
import random

mmlu_test_set = mmlu_dataset["test"]

indices = list(range(len(mmlu_test_set)))

random.seed(21)
random.shuffle(indices)

num_questions_to_select = 150
selected_indices = indices[:num_questions_to_select]
selected_mmlu_questions = mmlu_test_set.select(selected_indices)
selected_mmlu_questions

Dataset({
    features: ['question', 'subject', 'choices', 'answer'],
    num_rows: 150
})

In [ ]:
selected_mmlu_questions[0]

{'question': 'This question refers to the following information.\nPerestroika [Restructuring] is an urgent necessity arising from the profound processes of development in our socialist society. This society is ripe for change. It has long been yearning for it. Any delay in beginning perestroika could have led to an exacerbated internal situation in the near future, which, to put it bluntly, would have been fraught with serious social, economic, and political crises.\nMikhail Gorbachev, Perestroika: New Thinking for Our Country and the World, 1987\nFrom the passage, one may infer that Gorbachev believed that',
 'subject': 'high_school_european_history',
 'choices': ['the problems that required perestroika were the fault of capitalist enemies of socialism',
  'the problems that required perestroika were internal to the development of socialist society',
  'a socialist society could not work',
  'a socialist society could not coexist with capitalism'],
 'answer': 1}

**IMPORTANTE:** Troque "hf_YOUR_TOKEN_HERE" pelo seu próprio token do hugging face. Para ter acesso ao mistral, você precisa concordar com os termos no site

https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

In [ ]:
from huggingface_hub import login

# Substitua 'hf_YOUR_TOKEN_HERE' pelo seu token real do Hugging Face
login(token='hf_YOUR_TOKEN_HERE')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_8bit_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_8bit_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
answers = ['A', 'B', 'C', 'D']
text = f"""[INST]
You are an AI specialized in Multitask Language Understanding.
Based on a textual question, generate ONLY the answer to the question.
Do not write explanations.

Example 1
Q: {mmlu_dataset['dev'][0]['question']}
A. {mmlu_dataset['dev'][0]['choices'][0]}
B. {mmlu_dataset['dev'][0]['choices'][1]}
C. {mmlu_dataset['dev'][0]['choices'][2]}
D. {mmlu_dataset['dev'][0]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][0]['answer']]}

Example 2
Q: {mmlu_dataset['dev'][50]['question']}
A. {mmlu_dataset['dev'][50]['choices'][0]}
B. {mmlu_dataset['dev'][50]['choices'][1]}
C. {mmlu_dataset['dev'][50]['choices'][2]}
D. {mmlu_dataset['dev'][50]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][50]['answer']]}

Example 3
Q: {mmlu_dataset['dev'][100]['question']}
A. {mmlu_dataset['dev'][100]['choices'][0]}
B. {mmlu_dataset['dev'][100]['choices'][1]}
C. {mmlu_dataset['dev'][100]['choices'][2]}
D. {mmlu_dataset['dev'][100]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][100]['answer']]}

Example 4
Q: {mmlu_dataset['dev'][150]['question']}
A. {mmlu_dataset['dev'][150]['choices'][0]}
B. {mmlu_dataset['dev'][150]['choices'][1]}
C. {mmlu_dataset['dev'][150]['choices'][2]}
D. {mmlu_dataset['dev'][150]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][150]['answer']]}

New question
Q:
A.
B.
C.
D.
Answer:
[/INST]"""
print(text)

[INST]
You are an AI specialized in Multitask Language Understanding.
Based on a textual question, generate ONLY the answer to the question.
Do not write explanations.

Example 1
Q: Find all c in Z_3 such that Z_3[x]/(x^2 + c) is a field.
A. 0
B. 1
C. 2
D. 3
Answer: B

Example 2
Q: A refracting telescope consists of two converging lenses separated by 100 cm. The eye-piece lens has a focal length of 20 cm. The angular magnification of the telescope is
A. 4
B. 5
C. 6
D. 20
Answer: A

Example 3
Q: Which of the following is an example of the use of a device on the Internet of Things (IoT) ?
A. A car alerts a driver that it is about to hit an object.
B. A hiker uses a G P S watch to keep track of her position.
C. A refrigerator orders milk from an online delivery service when the milk in the refrigerator is almost gone.
D. A runner uses a watch with optical sensors to monitor his heart rate.
Answer: C

Example 4
Q: This question refers to the following information.
"Society in every state i

In [ ]:
def generate_response(question):
  text = f"""[INST]
You are an AI specialized in Multitask Language Understanding.
Based on a textual question, generate ONLY the answer to the question.
Do not write explanations. ONLY write the response letter.

Example 1
Q: {mmlu_dataset['dev'][0]['question']}
A. {mmlu_dataset['dev'][0]['choices'][0]}
B. {mmlu_dataset['dev'][0]['choices'][1]}
C. {mmlu_dataset['dev'][0]['choices'][2]}
D. {mmlu_dataset['dev'][0]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][0]['answer']]}

Example 2
Q: {mmlu_dataset['dev'][50]['question']}
A. {mmlu_dataset['dev'][50]['choices'][0]}
B. {mmlu_dataset['dev'][50]['choices'][1]}
C. {mmlu_dataset['dev'][50]['choices'][2]}
D. {mmlu_dataset['dev'][50]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][50]['answer']]}

Example 3
Q: {mmlu_dataset['dev'][100]['question']}
A. {mmlu_dataset['dev'][100]['choices'][0]}
B. {mmlu_dataset['dev'][100]['choices'][1]}
C. {mmlu_dataset['dev'][100]['choices'][2]}
D. {mmlu_dataset['dev'][100]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][100]['answer']]}

Example 4
Q: {mmlu_dataset['dev'][150]['question']}
A. {mmlu_dataset['dev'][150]['choices'][0]}
B. {mmlu_dataset['dev'][150]['choices'][1]}
C. {mmlu_dataset['dev'][150]['choices'][2]}
D. {mmlu_dataset['dev'][150]['choices'][3]}
Answer: {answers[mmlu_dataset['dev'][150]['answer']]}

New question
Q: {question['question']}
A. {question['choices'][0]}
B. {question['choices'][1]}
C. {question['choices'][2]}
D. {question['choices'][3]}
Answer:
[/INST]"""

  input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

  generated_ids = model.generate(
    input_ids,
    max_new_tokens=16,
    do_sample=True,
    temperature=0.01,
    pad_token_id=tokenizer.pad_token_id
  )

  full_response = tokenizer.decode(generated_ids[0], skip_special_tokens=False)

  #encontrando a resposta do modelo
  start_of_response_tag = "[/INST]"
  start_index = full_response.find(start_of_response_tag)

  try:
    # A resposta real começa logo após o [/INST] e um possível espaço.
    # Adicione o comprimento do tag e do espaço para pegar o início da resposta.
    model_raw_answer = full_response[start_index + len(start_of_response_tag):].strip()

    # O Mistral muitas vezes termina a resposta do modelo com </s>.
    # Remova esse token se ele estiver presente no final.
    if model_raw_answer.endswith("</s>"):
        model_answer = model_raw_answer[:-len("</s>")].strip()
    else:
        model_answer = model_raw_answer.strip()

    if "Answer:" in model_answer:
        model_answer = model_answer.split("Answer: ")[1].strip()

    model_answer = model_answer[0]

  except:
    model_answer = "Could not parse response."

  return model_answer

Calculando o score do modelo fine tunado (quando rodamos, o score foi 94/150)

In [ ]:
score = 0
for i in selected_mmlu_questions:
  response = generate_response(i)
  if response != answers[i['answer']]:
    print(i['question'])
    print(f"Predicted answer: {response}")
    print(f"Real answer: {answers[i['answer']]}\n")
  else:
    score += 1
    print(f"Correct! Score: {score}")

print(f"Final score: {score}/150")

Correct! Score: 1
Correct! Score: 2
Correct! Score: 3
Correct! Score: 4
Which of the following is true about dietary calcium:

Predicted answer: C
Real answer: D

Correct! Score: 5
Correct! Score: 6
Denise Developer found a near-perfect piece of property zoned commercial in the downtown area of Pleasantville. She hired an architect to design a high tech office building with an objective of attracting upscale tech-industry tenants. She approached Microhard and secured a lease agreement for over half of the available square footage. The lease was not recorded, but the agreement allowed Denise to attract other wealthy tenants. She also disclosed the Microhard lease on her application for a $1,500,000 construction loan from Maximum Mortgage Company. Denise executed a mortgage and related promissory note in favor of Maximum. Maximum recorded the mortgage immediately. The office building was completed and Denise had more tenant demand than available space, so the building opened full. One ye